# Exercícios de Análise com SQL
## U.S. EPA Food Commodity Intake Database (FCID)
### [https://fcid.foodrisk.org/](https://fcid.foodrisk.org/)

Ativando uma conexão de banco de dados em memória usando o SGBD H2:

In [1]:
%defaultDatasource jdbc:h2:mem:db

# Importando Tabelas do FCID

In [2]:
DROP TABLE IF EXISTS Crop_Group;
DROP TABLE IF EXISTS FCID_Description;
DROP TABLE IF EXISTS Recipes;
DROP TABLE IF EXISTS Intake;

CREATE TABLE Crop_Group (
  CGN VARCHAR(2),
  CGL VARCHAR(6),
  Crop_Group_Description VARCHAR(80),
  PRIMARY KEY (CGL)
) AS SELECT
  CGN, CGL, Crop_Group_Description
FROM CSVREAD('../../../data/food-intake/basics/FCID_Cropgroup_Description.csv');

CREATE TABLE FCID_Description (
  CGN VARCHAR(2),
  CG_Subgroup VARCHAR(6),
  FCID_Code VARCHAR(10),
  FCID_Desc VARCHAR(55),
  PRIMARY KEY (FCID_Code),
) AS SELECT
  cgn, CG_Subgroup, FCID_Code, FCID_Desc
FROM CSVREAD('../../../data/food-intake/basics/FCID_Code_Description.csv');

CREATE TABLE Recipes (
  Food_Code VARCHAR(8),
  Mod_Code VARCHAR(8),
  Ingredient_Num TINYINT,
  FCID_Code VARCHAR(10),
  Cooked_Status TINYINT,
  Food_Form TINYINT,
  Cooking_Method TINYINT,
  Commodity_Weight DECIMAL(5, 2),
  CSFII_9498_IND TINYINT,
  WWEIA_9904_IND TINYINT,
  WWEIA_0510_IND TINYINT,
  PRIMARY KEY(Food_Code, Mod_Code, Ingredient_Num),
  FOREIGN KEY(FCID_Code)
    REFERENCES FCID_Description(FCID_Code)
      ON DELETE NO ACTION
      ON UPDATE NO ACTION
) AS SELECT
  Food_Code, Mod_Code, Ingredient_Num, FCID_Code, Cooked_Status, Food_Form, Cooking_Method,
  Commodity_Weight, CSFII_9498_IND, WWEIA_9904_IND, WWEIA_0510_IND
FROM CSVREAD('../../../data/food-intake/recipes/Recipes_WWEIA_FCID_0510.csv');

CREATE TABLE Intake (
  SeqN INTEGER NOT NULL,
  DayCode TINYINT NOT NULL,
  DraBF TINYINT,
  FCID_Code VARCHAR(10),
  Cooked_Status TINYINT,
  Food_Form TINYINT,
  Cooking_Method TINYINT,
  Intake DECIMAL(13,7),
  Intake_BW DECIMAL(13,10),
  PRIMARY KEY(SeqN, DayCode, FCID_Code, Cooked_Status, Food_Form, Cooking_Method),
  FOREIGN KEY(FCID_Code)
    REFERENCES FCID_Description(FCID_Code)
      ON DELETE NO ACTION
      ON UPDATE NO ACTION
) AS SELECT
  SEQN, DAYCODE, DRABF, FCID_Code, Cooked_Status, Food_Form, Cooking_Method, Intake,Intake_BW
FROM CSVREAD('../../../data/food-intake/consumption/Commodity_CSFFM_Intake_0510-cropped.csv');

# Visualizando as Tabelas

## Grupos e Subgrupos de Cultura (Agrícolas)

Cada tupla é um grupo ou subgrupo. Quando o valor de `CGN` é igual a `CGL`, trata-se da descrição de um grupo, senão de um subgrupo.

| Campo | Descrição |
| --- | --- |
| CGN | código do grupo de cultura (agrícola) |
| CGL | código do subgrupo de cultura (agrícola) |
| Crop_Group_Description | descrição do grupo ou subgrupo de cultura |

In [3]:
SELECT * FROM Crop_Group LIMIT 10;

## Ingredientes (Produtos Alimentares de Base)

Cada tupla é um produto alimentar de base usado em receitas.

| Campo | Descrição | Chave para a Tabela |
| --- | --- | --- |
| CGN | código do grupo de cultura (agrícola) | associado à tabela de Grupos e Subgrupos de Cultura |
| CG_Subgroup | código do subgrupo de cultura (agrícola) | associado à tabela de Grupos e Subgrupos de Cultura |
| FCID_Code | código do ingrediente | |
| FCID_Desc | descrição do ingrediente | |

In [4]:
SELECT * FROM FCID_Description LIMIT 10;

## Receitas - Composição

Cada tupla é um item que é parte de uma receita. Cada receita - todas as tuplas com o mesmo `Food_code`.

| Campo | Descrição | Chave para a Tabela |
| --- | --- | --- |
| Food_Code | código da receita | Alimentos feitos por Receitas |
| Mod_Code | código de receita modificada (0 = sem modificação) | |
| Ingredient_Num | número do ingrediente dentro da receita | |
| FCID_Code | código do ingrediente | Ingredientes (Produtos Alimentares) |
| Cooked_Status | estado e cozimento original do ingrediente | Estado de Cozimento do Produto |
| Food_Form | forma alimentar | Forma do Produto Alimentar |
| Cooking_Method | método de cozimento do ingrediente na receita | Método de Cozimento do Produto na Receita |
| Commodity_Weight | participação (em percentual) do ingrediente no peso da receita | |

In [5]:
SELECT * FROM Recipes LIMIT 10;

## Consumo

Cada tupla é um produto alimentar (ingrediente) ingerido por uma pessoa em um dia. Todas as tuplas com mesmo `SEQN` são ingestões da mesma pessoa no dia 1 ou 2 (`DAYCODE`).

| Campo | Descrição | Chave para a Tabela ou Valor |
| --- | --- | --- |
| SEQN  | id do entrevistado | |
| DRABF | indica se o entrevistado (bebê) foi amamentado em qualquer um dos dois dias de retorno | 1 = sim; 2 = não |
| DAYCODE | Dia do diário alimentar. | 1 = dia 1; 2= dia 2 |
| FCID_Code | código do produto alimentar (usado como ingrediente) | Ingredientes (Produtos Alimentares) |
| Cooked_Status | estado e cozimento original do ingrediente | Estado de Cozimento do Produto |
| Food_Form | forma alimentar | Forma do Produto Alimentar |
| Cooking_Method | método de cozimento do ingrediente na receita | Método de Cozimento do Produto na Receita |
| Intake | ingestão. em gramas | |
| Intake_BW | ingestão. em gramas por quilograma de peso corporal (g/kg pc) | |

In [6]:
SELECT * FROM Intake LIMIT 10;

## 1)  Apresentando uma Receita

* Liste os ingredientes da receita de código `27111300` - Mexican style beef stew, no potatoes, tomato-based sauce (mixture).
* Não devem aparecer as modificações da receita.
* Mostre apenas o código da receita, o código de cada ingrediente, sua ordem e a participação no peso.

## 2) Receita com Nomes de Ingredientes
* Aprimore a solução (1) para apresentar o nome dos ingredientes junto com seus códigos.

## 3) Grupos Alimentares da Receita
* A partir da receita escolhida em (1), apresente os grupos alimentares dos ingredientes contidos na receita.
* Cada grupo alimentar só deve aparecer uma vez no resultado.
* Para se obter o nome do grupo alimentar na tabela `Crop_Group` (não o subgrupo) devem ser considerados os registros em que a coluna `CGN` é igual à `CGL`.

## 4) Participação nas Receitas
* Liste o nome de cada um dos produtos alimentares seguido do número de receitas em que ele participa.
* A lista deve ser apresentada em ordem decrescente de número de participação em receitas.

## 5) Consumo Médio de Grupos de Alimentos
* Para cada Grupo de Alimentos (cada CGN diferente na tabela Crop_Group), apresente sua descrição e o consumo médio deste grupo de alimentos.
* O consumo é definido pelo campo `Intake` da tabela `Intake`.